In [8]:
import pandas as pd
from pathlib import Path

DATA_PATH = 'data'

file_name = (
    Path('tables_OZ_geo_5500') /
    'Ozon_Crawler_Latest_info2025-04-07-12-57-51.xlsx'
)

source_df = pd.read_excel(Path(DATA_PATH) / file_name)
source_df.head(1)

,Sku (Sku),Фото (CoverImage),Название товара (ProductName),Продавец (SellerName),Бренд (BrandName),Название категории (CategoryName),Цена соинвест (DiscountPrice),Цена по карте (OzonCardPrice),Сток FBO (StockFbm),Сток FBS (StockFbs),Ошибка загрузки (CrawlerError),Валюта (Currency),Цена до скидки (BasePrice),Рейтинг товара (AvgRating),Количество отзывов (Reviews),Описание (Description),Rich-контент (RichContent),Ссылка на карточку,SellerProductId (SellerProductId)
0,1871769771,https://cdn1.ozone.ru/s3/multimedia-1-b/732378...,"Карты МИРА и РОССИИ настенные политические,160...",ИНТЕРТРЕЙД,Атлас Принт,Карты мира,1785.0,1649,0,34,0,RUB,3867.0,5.0,1,Представляем вашему вниманию уникальный набор ...,NaN,https://www.ozon.ru/product/karty-mira-i-rossi...,191942573


In [9]:
# Collect images

import nest_asyncio
nest_asyncio.apply()

from utils import parallel_download_img

# Dataset name
DATASET_NAME = "images_OZ_geo_5500"

# Download image files using the 'Thumb' column for both URL and ID extraction
results = await parallel_download_img(
    df=source_df,
    url_col='Фото (CoverImage)',
    img_id_regex=r'/(\d+)\.jpg$',  # capture numeric ID before '.jpg'
    dataset_name=DATASET_NAME,
    base_download_dir=DATA_PATH,
    max_concurrency=100,
    delay=1.0
)

In [12]:
# Merge the list of results into your DataFrame as a new column
source_df['image_name'] = results

In [15]:
# Save images and names

file_path = (
    Path(DATA_PATH) /
    Path('tables_OZ_geo_5500') /
    Path('OZ_geo_5500.csv')
)

target_df = (
    source_df[[
        'Sku (Sku)',
        'Название товара (ProductName)',
        'Описание (Description)',
        'Ссылка на карточку',
        'Продавец (SellerName)',
        'image_name'
    ]]
    .rename(columns={
        'Sku (Sku)': 'SKU',
        'Название товара (ProductName)': 'name',
        'Описание (Description)': 'description',
        'Ссылка на карточку': 'URL',
        'Продавец (SellerName)': 'Seller'
    })
)

target_df.to_csv(file_path, index=None)